In [1]:
import os

os.sys.path.insert(0, '../script')

import webnlg

In [2]:
webnlg.DATASETS_FILEPATHS.keys()

dict_keys(['dev', 'train', 'test_no_lex', 'test_unseen_with_lex', 'test_with_lex', 'dev_1.2', 'train_1.2'])

In [3]:
corpus = webnlg.load(dataset=['dev', 'train', 'test_with_lex'])

In [21]:
sample = corpus.sample(seed=300)

sample

Triple info: category=Food eid=Id213

	Modified triples:

Philippines | ethnicGroup | Ilocano_people
Philippines | language | Arabic
Philippines | ethnicGroup | Zamboangans
Philippines | language | Philippine_Spanish
Batchoy | country | Philippines


	Lexicalizations:

Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups.


Zamboangans and Ilocano people are ethnic groups from the Philippines which is home to the dish batchoy and where Philippine Spanish and Arabic are spoken.


Batchoy comes from the Philippines, where Philippine Spanish and Arabic are among the languages spoken, and the Zamboangans and IIocano people are ethnic groups.


In [25]:
def pairing(iterable):
    
    for i, item1 in enumerate(iterable):
        
        for j, item2 in enumerate(iterable):
            
            if i != j:
                
                yield (item1, item2)

In [26]:
list(pairing(sample.lexes()))

[('Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups.',
  'Zamboangans and Ilocano people are ethnic groups from the Philippines which is home to the dish batchoy and where Philippine Spanish and Arabic are spoken.'),
 ('Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups.',
  'Batchoy comes from the Philippines, where Philippine Spanish and Arabic are among the languages spoken, and the Zamboangans and IIocano people are ethnic groups.'),
 ('Zamboangans and Ilocano people are ethnic groups from the Philippines which is home to the dish batchoy and where Philippine Spanish and Arabic are spoken.',
  'Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups

In [29]:
with open('paraphrases.txt', 'w') as f:
    
    f.write('input,output\n')
    
    i = 0
    
    for entry in corpus:
        
        for pair in pairing(entry.lexes()):
            
            f.write('{},{}\n'.format(*pair))
            i += 1

# Delexicalized

In [42]:
delex = webnlg.load(dataset=['dev_1.2', 'train_1.2'])

with open('paraphrases_delex.txt', 'w') as f:
    
    f.write('input,output\n')
    
    i = 0
    
    for entry in delex:
        
        for pair in pairing([l['template'] for l in entry.entry['lexes']]):
            
            f.write('{},{}\n'.format(*pair))
            i += 1

In [43]:
!head paraphrases_delex.txt

input,output
AGENT-1 was created by PATIENT-1 .,The creator of AGENT-1 is PATIENT-1 .
AGENT-1 was created by PATIENT-1 .,AGENT-1 , was created by PATIENT-1 .
The creator of AGENT-1 is PATIENT-1 .,AGENT-1 was created by PATIENT-1 .
The creator of AGENT-1 is PATIENT-1 .,AGENT-1 , was created by PATIENT-1 .
AGENT-1 , was created by PATIENT-1 .,AGENT-1 was created by PATIENT-1 .
AGENT-1 , was created by PATIENT-1 .,The creator of AGENT-1 is PATIENT-1 .
AGENT-1 was created by PATIENT-1 .,AGENT-1 was created by PATIENT-1 .
AGENT-1 was created by PATIENT-1 .,AGENT-1 was created by PATIENT-1 .
AGENT-1 was created by PATIENT-1 .,AGENT-1 was created by PATIENT-1 .
